In [ ]:
import nbtest
import json
import numpy as np
import matplotlib.pyplot as plt
random_seed = np.random.randint(10000)

# Data Understanding Step

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('./heart_failure.csv')
nbtest.instrument(df, 3, 3, 'heart-failure-classification', 'df', api='read_csv')

In [ ]:
nbtest_tmpvar_0 = df.head(n=10)
nbtest.instrument(nbtest_tmpvar_0, 4, 1, 'heart-failure-classification', 'nbtest_tmpvar_0', api='head')

In [ ]:
nbtest_tmpvar_1 = df.info()
nbtest.instrument(nbtest_tmpvar_1, 5, 1, 'heart-failure-classification', 'nbtest_tmpvar_1', api='info')

In [ ]:
nbtest_tmpvar_2 = df.isnull().any()
nbtest.instrument(nbtest_tmpvar_2, 6, 1, 'heart-failure-classification', 'nbtest_tmpvar_2', api='any')

In [ ]:
df.shape
nbtest.instrument(df.shape, 7, 1, 'heart-failure-classification', 'df.shape')

In [ ]:
nbtest_tmpvar_3 = df.describe()
nbtest.instrument(nbtest_tmpvar_3, 8, 1, 'heart-failure-classification', 'nbtest_tmpvar_3', api='describe')

In [ ]:
for i in df.columns:
    print(i)
    nbtest.instrument(i, 9, 2, 'heart-failure-classification', 'i')
    print(df[f'{i}'].value_counts())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.pairplot(df, vars=['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time', 'DEATH_EVENT'], hue='DEATH_EVENT')
nbtest_tmpvar_4 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_4, 10, 6, 'heart-failure-classification', 'nbtest_tmpvar_4')
plt.show()

In [ ]:
korelasyon_matrisi = df.corr().abs()
plt.figure(figsize=(8, 6), dpi=100)
nbtest_tmpvar_5 = sns.heatmap(korelasyon_matrisi, annot=True, cmap='rocket_r', fmt='.2f', linewidths=1)
nbtest.instrument(nbtest_tmpvar_5, 11, 3, 'heart-failure-classification', 'nbtest_tmpvar_5', api='heatmap')
plt.title('Korelasyon Heatmap')
nbtest_tmpvar_6 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_6, 11, 5, 'heart-failure-classification', 'nbtest_tmpvar_6')
plt.show()

In [ ]:
print(df.corr().abs()['DEATH_EVENT'].nlargest(15))

In [ ]:
col_reduced = df.corr().abs()['DEATH_EVENT'].nlargest(6).index
col_reduced

# **Data Slicing Step**

In [ ]:
x = df[['time', 'serum_creatinine', 'ejection_fraction', 'age', 'serum_sodium']].values
y = df.iloc[:, -1].values

# **Model Training**

# **Train and Test Split**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=random_seed)
nbtest.instrument(y_test, 18, 2, 'heart-failure-classification', 'y_test', api='train_test_split')
nbtest.instrument(y_train, 18, 2, 'heart-failure-classification', 'y_train', api='train_test_split')
nbtest.instrument(x_test, 18, 2, 'heart-failure-classification', 'x_test', api='train_test_split')
nbtest.instrument(x_train, 18, 2, 'heart-failure-classification', 'x_train', api='train_test_split')

#**Verilerin Ölceklenmesi**

In [ ]:
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

#**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression(random_state=random_seed)
logr.fit(X_train, y_train)
y_pred = logr.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=logr.classes_)
nbtest.instrument(cm, 22, 5, 'heart-failure-classification', 'cm', api='confusion_matrix')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=logr.classes_)
disp.plot(cmap='YlGnBu')
nbtest_tmpvar_7 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_7, 22, 8, 'heart-failure-classification', 'nbtest_tmpvar_7')
plt.show()
print()
print('accuracy: ', accuracy_score(y_test, y_pred))
nbtest.instrument(accuracy_score(y_test, y_pred), 22, 10, 'heart-failure-classification', 'accuracy_score(y_test, y_pred)', api='accuracy_score')

In [ ]:
import sklearn
y_proba = logr.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)
nbtest.instrument(roc_auc, 23, 4, 'heart-failure-classification', 'roc_auc', api='auc')
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
nbtest_tmpvar_8 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_8, 23, 14, 'heart-failure-classification', 'nbtest_tmpvar_8')
plt.show()
print(sklearn.metrics.roc_auc_score(y_test, y_pred))
nbtest.instrument(sklearn.metrics.roc_auc_score(y_test, y_pred), 23, 15, 'heart-failure-classification', 'sklearn.metrics.roc_auc_score(y_test, y_pred)', api='roc_auc_score')

#**KNN - K Nearest Neighboor**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=knn.classes_)
nbtest.instrument(cm, 25, 5, 'heart-failure-classification', 'cm', api='confusion_matrix')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=knn.classes_)
disp.plot(cmap='YlGnBu')
nbtest_tmpvar_9 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_9, 25, 8, 'heart-failure-classification', 'nbtest_tmpvar_9')
plt.show()
print()
print('accuracy: ', accuracy_score(y_test, y_pred))
nbtest.instrument(accuracy_score(y_test, y_pred), 25, 10, 'heart-failure-classification', 'accuracy_score(y_test, y_pred)', api='accuracy_score')

In [ ]:
import matplotlib.pyplot as plt
y_proba = knn.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)
nbtest.instrument(roc_auc, 26, 4, 'heart-failure-classification', 'roc_auc', api='auc')
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
nbtest_tmpvar_10 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_10, 26, 14, 'heart-failure-classification', 'nbtest_tmpvar_10')
plt.show()
print(sklearn.metrics.roc_auc_score(y_test, y_pred))
nbtest.instrument(sklearn.metrics.roc_auc_score(y_test, y_pred), 26, 15, 'heart-failure-classification', 'sklearn.metrics.roc_auc_score(y_test, y_pred)', api='roc_auc_score')

#**SVC - Support Vector Classification**

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf', probability=True)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=svc.classes_)
nbtest.instrument(cm, 28, 5, 'heart-failure-classification', 'cm', api='confusion_matrix')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=svc.classes_)
disp.plot(cmap='YlGnBu')
nbtest_tmpvar_11 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_11, 28, 8, 'heart-failure-classification', 'nbtest_tmpvar_11')
plt.show()
print()
print('accuracy: ', accuracy_score(y_test, y_pred))
nbtest.instrument(accuracy_score(y_test, y_pred), 28, 10, 'heart-failure-classification', 'accuracy_score(y_test, y_pred)', api='accuracy_score')

In [ ]:
import matplotlib.pyplot as plt
y_proba = svc.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)
nbtest.instrument(roc_auc, 29, 4, 'heart-failure-classification', 'roc_auc', api='auc')
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
nbtest_tmpvar_12 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_12, 29, 14, 'heart-failure-classification', 'nbtest_tmpvar_12')
plt.show()
print(sklearn.metrics.roc_auc_score(y_test, y_pred))
nbtest.instrument(sklearn.metrics.roc_auc_score(y_test, y_pred), 29, 15, 'heart-failure-classification', 'sklearn.metrics.roc_auc_score(y_test, y_pred)', api='roc_auc_score')

#**Naive Bayes - Gaussian**

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=gnb.classes_)
nbtest.instrument(cm, 31, 5, 'heart-failure-classification', 'cm', api='confusion_matrix')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gnb.classes_)
disp.plot(cmap='YlGnBu')
nbtest_tmpvar_13 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_13, 31, 8, 'heart-failure-classification', 'nbtest_tmpvar_13')
plt.show()
print()
print('accuracy: ', accuracy_score(y_test, y_pred))
nbtest.instrument(accuracy_score(y_test, y_pred), 31, 10, 'heart-failure-classification', 'accuracy_score(y_test, y_pred)', api='accuracy_score')

In [ ]:
import matplotlib.pyplot as plt
y_proba = gnb.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)
nbtest.instrument(roc_auc, 32, 4, 'heart-failure-classification', 'roc_auc', api='auc')
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
nbtest_tmpvar_14 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_14, 32, 14, 'heart-failure-classification', 'nbtest_tmpvar_14')
plt.show()
print(sklearn.metrics.roc_auc_score(y_test, y_pred))
nbtest.instrument(sklearn.metrics.roc_auc_score(y_test, y_pred), 32, 15, 'heart-failure-classification', 'sklearn.metrics.roc_auc_score(y_test, y_pred)', api='roc_auc_score')

#**Decision Tree Classifier**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion='entropy', random_state=random_seed)
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=dtc.classes_)
nbtest.instrument(cm, 34, 5, 'heart-failure-classification', 'cm', api='confusion_matrix')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dtc.classes_)
disp.plot(cmap='YlGnBu')
nbtest_tmpvar_15 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_15, 34, 8, 'heart-failure-classification', 'nbtest_tmpvar_15')
plt.show()
print()
print('accuracy: ', accuracy_score(y_test, y_pred))
nbtest.instrument(accuracy_score(y_test, y_pred), 34, 10, 'heart-failure-classification', 'accuracy_score(y_test, y_pred)', api='accuracy_score')

In [ ]:
import matplotlib.pyplot as plt
y_proba = dtc.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)
nbtest.instrument(roc_auc, 35, 4, 'heart-failure-classification', 'roc_auc', api='auc')
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
nbtest_tmpvar_16 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_16, 35, 14, 'heart-failure-classification', 'nbtest_tmpvar_16')
plt.show()
print(sklearn.metrics.roc_auc_score(y_test, y_pred))
nbtest.instrument(sklearn.metrics.roc_auc_score(y_test, y_pred), 35, 15, 'heart-failure-classification', 'sklearn.metrics.roc_auc_score(y_test, y_pred)', api='roc_auc_score')

#**Random Forest Classification** - Best Model ✅

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=35, criterion='entropy', random_state=random_seed)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=rfc.classes_)
nbtest.instrument(cm, 37, 5, 'heart-failure-classification', 'cm', api='confusion_matrix')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rfc.classes_)
disp.plot(cmap='YlGnBu')
nbtest_tmpvar_17 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_17, 37, 8, 'heart-failure-classification', 'nbtest_tmpvar_17')
plt.show()
print()
print('accuracy: ', accuracy_score(y_test, y_pred))
nbtest.instrument(accuracy_score(y_test, y_pred), 37, 10, 'heart-failure-classification', 'accuracy_score(y_test, y_pred)', api='accuracy_score')

In [ ]:
import matplotlib.pyplot as plt
y_proba = rfc.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)
nbtest.instrument(roc_auc, 38, 4, 'heart-failure-classification', 'roc_auc', api='auc')
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
nbtest_tmpvar_18 = plt.gcf()
nbtest.instrument(nbtest_tmpvar_18, 38, 14, 'heart-failure-classification', 'nbtest_tmpvar_18')
plt.show()
print(sklearn.metrics.roc_auc_score(y_test, y_pred))
nbtest.instrument(sklearn.metrics.roc_auc_score(y_test, y_pred), 38, 15, 'heart-failure-classification', 'sklearn.metrics.roc_auc_score(y_test, y_pred)', api='roc_auc_score')

In [ ]:
exit()